### Week 6:

Train a multi-class classification model on AutoML.    

https://cloud.google.com/blog/products/ai-machine-learning/use-automl-tables-from-a-jupyter-notebook 

In [4]:
#from google.cloud import automl
from google.cloud import automl_v1beta1

In [6]:
client = automl_v1beta1.TablesClient(project='msds-434-robords-oct')

/Users/robords/.venv/autoML/lib/python3.7/site-packages/google/auth/_default.py:83: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/robords/.venv/autoML/lib/python3.7/site-packages/google/auth/_default.py:83: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about 